In [35]:
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [36]:
df = sns.load_dataset('titanic')

In [37]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [38]:
x=df.iloc[:,1:8]
y=df.iloc[:,0:1]

In [39]:
x

,pclass,sex,age,sibsp,parch,fare,embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,S
887,1,female,19.0,0,0,30.0000,S
888,3,female,NaN,1,2,23.4500,S
889,1,male,26.0,0,0,30.0000,C


In [40]:
y

,survived
0,0
1,1
2,1
3,1
4,0
...,...
886,0
887,1
888,0
889,1


In [41]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42,test_size=0.2)

In [42]:
x_train

,pclass,sex,age,sibsp,parch,fare,embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [43]:
x_train.isnull().sum()

pclass        0
sex           0
age         140
sibsp         0
parch         0
fare          0
embarked      2
dtype: int64

In [44]:
x_train['embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [45]:
print(x_train['sex'].unique())


['male' 'female']


In [46]:
import numpy as np
simpleimputer_ct1 = ColumnTransformer(transformers=[
    ('meanimputer_age',SimpleImputer(strategy='mean'),[2]),
    ('mostfrequent_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [47]:
ohe_ct2 = ColumnTransformer(transformers=[
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='error'),[1,3])
],remainder='passthrough')

In [48]:
scaler_ct3= ColumnTransformer(transformers=[
    ('scaler',MinMaxScaler(),slice(0,10))
],remainder='passthrough')

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# model_name = DecisionTreeClassifier(random_state=0)
# model_name = LogisticRegression(random_state=0)
model_name = SVC(random_state=0)

### In model we tried decision tree and logistic regression but logistic regression accuracy was slightly higher so we used logistic regression

In [50]:
x_train.columns

Index(['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked'], dtype='object')

# Create pipeline


In [51]:
pipe = Pipeline([
    ('simpleimputer_ct1',simpleimputer_ct1),
    ('ohe_ct2',ohe_ct2),
    ('scaler_ct3',scaler_ct3),
    ('model',model_name)
])

In [52]:
pipe

Pipeline(steps=[('simpleimputer_ct1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('meanimputer_age',
                                                  SimpleImputer(), [2]),
                                                 ('mostfrequent_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('ohe_ct2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(sparse=False),
                                                  [1, 3])])),
                ('scaler_ct3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('model', SVC(random_state=0))])

In [53]:
pipe.fit(x_train,y_train)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('simpleimputer_ct1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('meanimputer_age',
                                                  SimpleImputer(), [2]),
                                                 ('mostfrequent_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('ohe_ct2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(sparse=False),
                                                  [1, 3])])),
                ('scaler_ct3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('model', SVC(random_state=0))])

In [54]:
# df1 = pd.DataFrame(pipe_result)


In [55]:
x_train.head(10)

,pclass,sex,age,sibsp,parch,fare,embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
118,1,male,24.0,0,1,247.5208,C
536,1,male,45.0,0,0,26.5500,S
361,2,male,29.0,1,0,27.7208,C
29,3,male,NaN,0,0,7.8958,S
55,1,male,NaN,0,0,35.5000,S


In [56]:
#0.emb_c  1.emb_q 2.emb_s 3.ge_f 4.ge_m 5.age 6.pclass 7.sibsp 8.parch 9.fare 
# df1.head(10)

In [57]:
# to see all the steps done through pipeline
pipe.named_steps

{'simpleimputer_ct1': ColumnTransformer(remainder='passthrough',
                   transformers=[('meanimputer_age', SimpleImputer(), [2]),
                                 ('mostfrequent_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'ohe_ct2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(sparse=False), [1, 3])]),
 'scaler_ct3': ColumnTransformer(remainder='passthrough',
                   transformers=[('scaler', MinMaxScaler(), slice(0, 10, None))]),
 'model': SVC(random_state=0)}

In [58]:
# To access specific step from pipeline
pipe.named_steps['simpleimputer_ct1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('meanimputer_age', SimpleImputer(), [2]),
                                ('mostfrequent_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [59]:
x_train

,pclass,sex,age,sibsp,parch,fare,embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


## Making prediction on test data

In [60]:
y_predict = pipe.predict(x_test)

In [61]:
y_predict

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0], dtype=int64)

## Calculating the accuracy of the model

In [62]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.7932960893854749

## Cross Validation using Pipeline

#### In cross validation cv=5 paramater means for 5 times make random train test split then run the pipeline (preprocessing, exexuting the model) then calculate the mean of accuracy for 5 times

In [63]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,x_train,y_train,cv=5,scoring='accuracy').mean()

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y 

0.8160149709445484

## Hyper paramater tuining uing GridSearch CV using Pipeline

In [64]:
params=[{       
   'model__kernel':['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
    'model__gamma':['scale', 'auto'],
    'model__C':np.logspace(-2,2,10),
    'model__max_iter':[100,1000,2500,5000]
}
]


In [65]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5,verbose=True)
grid.fit(x_train, y_train)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its de

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('simpleimputer_ct1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('meanimputer_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('mostfrequent_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('ohe_ct2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(sparse=False),
                                                                         [...
                                       ('model', SVC(random_state=0))]),
             param_grid=[{'model__C': array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02]),
                          'model__gamma': ['scale', 'auto'],
                          'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid',
                                            'precomputed'],
                          'model__max_iter': [100, 1000, 2500, 5000]}],
             verbose=True)

In [66]:
grid.best_score_

0.8258347286516301

In [ ]:
grid.best_params_